# 프로젝트 - Movielens 영화 추천 

### Movielens Data 

- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
- 별점을 시청횟수로 해석해서 생각하겠습니다.
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

## 1) 데이터 준비와 전처리

In [1]:
import os
import pandas as pd
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
from implicit.als import AlternatingLeastSquares
import os
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'
from scipy.sparse import csr_matrix



In [2]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# rating 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'rating':'counts'}, inplace=True)

In [5]:
ratings.head()

,user_id,movie_id,counts,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


### data set merge

In [7]:
movies_ratings = pd.merge(ratings, movies, on="movie_id")
movies_ratings.head()

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama


## 2) 분석해 봅시다.

- ratings에 있는 유니크한 영화 개수
- rating에 있는 유니크한 사용자 수
- 가장 인기있는 영화 30개(인기순)

In [8]:
# 영화 갯수
ratings['movie_id'].nunique()

3628

In [9]:
# 유저 수 
ratings['user_id'].nunique()

6039

In [10]:
# 첫 번쨰 유저가 어떤 영화 보는지
condition = movies_ratings['user_id']== movies_ratings.loc[0, 'user_id']
movies_ratings.loc[condition]

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1680,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2123,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
2734,1,3408,4,978300275,Erin Brockovich (2000),Drama
3957,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy
5556,1,1197,3,978302268,"Princess Bride, The (1987)",Action|Adventure|Comedy|Romance
7808,1,1287,5,978302039,Ben-Hur (1959),Action|Adventure|Drama
8474,1,2804,5,978300719,"Christmas Story, A (1983)",Comedy|Drama
9764,1,594,4,978302268,Snow White and the Seven Dwarfs (1937),Animation|Children's|Musical
10471,1,919,4,978301368,"Wizard of Oz, The (1939)",Adventure|Children's|Drama|Musical


In [11]:
# 가장 인기 있는 영화 30개 
movie_count = movies_ratings.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [12]:
# 유저별 counts  중앙값에 대한 통계
user_median = movies_ratings.groupby('user_id')['counts'].median()
user_median.describe()

count    6039.000000
mean        4.055970
std         0.432143
min         3.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         5.000000
Name: counts, dtype: float64

## 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가

In [13]:
my_favorite = ['Pulp Fiction (1994)' , 'Silence of the Lambs, The (1991)' ,'Total Recall (1990)' ,'Matrix, The (1999)' ,'Godfather, The (1972)']

# jang 이라는 user_id 위 영화를 5번씩 들었다고 가정 
my_playlist = pd.DataFrame({'user_id': ['jang']*5, 'title': my_favorite, 'counts':[5]*5})

if not movies_ratings.isin({'user_id':['zimin']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    movies_ratings = movies_ratings.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

movies_ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,counts,timestamp,title,genre
836473,5851,3607.0,5,9.577566e+08,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026.0,4,9.583469e+08,Slaughterhouse (1987),Horror
836475,5854,690.0,3,9.577443e+08,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909.0,4,9.572734e+08,"Five Wives, Three Secretaries and Me (1998)",Documentary
836477,5948,1360.0,5,1.016564e+09,Identification of a Woman (Identificazione di ...,Drama
0,jang,NaN,5,NaN,Pulp Fiction (1994),NaN
1,jang,NaN,5,NaN,"Silence of the Lambs, The (1991)",NaN
2,jang,NaN,5,NaN,Total Recall (1990),NaN
3,jang,NaN,5,NaN,"Matrix, The (1999)",NaN
4,jang,NaN,5,NaN,"Godfather, The (1972)",NaN


## 4) CSR matrix를 직접 만들어 봅시다.

###  data 전처리 
- indexing

In [14]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = movies_ratings['user_id'].unique()
movies_unique = movies_ratings['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movies_to_idx = {v:k for k,v in enumerate(movies_unique)}

In [15]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['jang'])  
print(movies_to_idx['Pulp Fiction (1994)'])

6039
222


In [16]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다.

temp_user_data = movies_ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(movies_ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    movies_ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = movies_ratings['title'].map(movies_to_idx.get).dropna()
if len(temp_movie_data) == len(movies_ratings):
    print('movie column indexing OK!!')
    movies_ratings['title'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

movies_ratings

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,counts,timestamp,title,genre
0,0,1193.0,5,978300760.0,0,Drama
1,1,1193.0,5,978298413.0,0,Drama
2,2,1193.0,4,978220179.0,0,Drama
3,3,1193.0,4,978199279.0,0,Drama
4,4,1193.0,5,978158471.0,0,Drama
...,...,...,...,...,...,...
0,6039,NaN,5,NaN,222,NaN
1,6039,NaN,5,NaN,121,NaN
2,6039,NaN,5,NaN,62,NaN
3,6039,NaN,5,NaN,124,NaN


### CSR(Compressed Sparse Row) Matrix

In [17]:
num_user = movies_ratings['user_id'].nunique()
num_movie = movies_ratings['title'].nunique()

csr_data = csr_matrix((movies_ratings.counts, (movies_ratings.user_id, movies_ratings.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5) AlternatingLeastSquares 모델을 구성 / 훈련

In [18]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, 
                                    regularization=0.01, 
                                    use_gpu=False, 
                                    iterations=15, 
                                    dtype=np.float32)

In [19]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

### 훈련 

In [20]:
# 모델 훈련
als_model.fit(csr_data_transpose)

## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [21]:
jang, matrix = user_to_idx['jang'], movies_to_idx['Matrix, The (1999)']

jang_vector, matrix_vector = als_model.user_factors[jang], als_model.item_factors[matrix]


### jang/ matrix 내적화 

In [23]:
np.dot(jang_vector, matrix_vector)

0.5333452

In [25]:
pulp_fiction = movies_to_idx['Pulp Fiction (1994)']
pulp_fiction_vector = als_model.item_factors[pulp_fiction]
np.dot(jang_vector, pulp_fiction_vector)

0.51221395

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [30]:
favorite_movie = 'Total Recall (1990)'
movie_id = movies_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N= 15)
similar_movie

[(62, 0.16757666),
 (92, 0.119589366),
 (124, 0.11188692),
 (175, 0.10066117),
 (107, 0.09645954),
 (375, 0.09439388),
 (145, 0.09316504),
 (200, 0.09137084),
 (179, 0.08609518),
 (865, 0.08064149),
 (75, 0.08042923),
 (236, 0.07663409),
 (141, 0.07637513),
 (377, 0.07334481),
 (713, 0.070815355)]

### movie id를 다시 영화 이름으로 맵핑

In [31]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_movie= {v:k for k,v in movies_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Total Recall (1990)',
 'Terminator 2: Judgment Day (1991)',
 'Matrix, The (1999)',
 'Men in Black (1997)',
 'Jurassic Park (1993)',
 'Face/Off (1997)',
 'Fifth Element, The (1997)',
 'Terminator, The (1984)',
 'Rock, The (1996)',
 'Predator (1987)',
 'Hunt for Red October, The (1990)',
 'Speed (1994)',
 'Fugitive, The (1993)',
 'In the Line of Fire (1993)',
 'Starship Troopers (1997)']

In [34]:
def get_similar_movie(movie_name: str):
    movie_id = movies_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [35]:
get_similar_movie('Starship Troopers (1997)')

['Starship Troopers (1997)',
 'Stargate (1994)',
 'Mars Attacks! (1996)',
 'Deep Impact (1998)',
 'Fifth Element, The (1997)',
 'Independence Day (ID4) (1996)',
 'Strange Days (1995)',
 'Demolition Man (1993)',
 'Species (1995)',
 'Spawn (1997)']

## 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [36]:
user = user_to_idx['jang']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(380, 0.61215496),
 (51, 0.55919117),
 (92, 0.54486036),
 (157, 0.4946834),
 (269, 0.44925216),
 (233, 0.4218248),
 (200, 0.37030166),
 (175, 0.33446336),
 (224, 0.33297896),
 (23, 0.32754517),
 (38, 0.32373872),
 (435, 0.31741568),
 (317, 0.30392504),
 (141, 0.28904504),
 (107, 0.28361547),
 (289, 0.27849886),
 (48, 0.2629677),
 (375, 0.24824736),
 (248, 0.22097887),
 (220, 0.21963727)]

In [37]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Godfather: Part II, The (1974)',
 'Fargo (1996)',
 'Terminator 2: Judgment Day (1991)',
 'Shawshank Redemption, The (1994)',
 'GoodFellas (1990)',
 'Usual Suspects, The (1995)',
 'Terminator, The (1984)',
 'Men in Black (1997)',
 'L.A. Confidential (1997)',
 "Schindler's List (1993)",
 'Sixth Sense, The (1999)',
 'Godfather: Part III, The (1990)',
 'Twelve Monkeys (1995)',
 'Fugitive, The (1993)',
 'Jurassic Park (1993)',
 'Reservoir Dogs (1992)',
 'Saving Private Ryan (1998)',
 'Face/Off (1997)',
 'Good Will Hunting (1997)',
 'Seven (Se7en) (1995)']